**Tutorial of converting TF-ckpt to Pytorch model**

**Environment:**
- Anaconda
- Python: Python 3.7.13
- Tensorflow-gpu: 2.3.0
- cudnn: 7.6.5
- transformers: 4.18.0
- cudatoolkit: 10.2.89 (will be installed when installing pytorch )



In [ ]:
# Error loading "D:\Xie\anaconda3\envs\model_convert\lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies. 
conda install pytorch==1.10.1 torchvision==0.11.2 torchaudio==0.10.1 cudatoolkit=10.2 -c pytorch

In [ ]:
# Install Pytorch 
pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# Test Pytorch, create [test_pytorch.py] with the following code:
import torch
x = torch.rand(5, 3)
print(x)

In [ ]:
python test_pytorch.py

- Results will be as follow if successed:
      tensor([[0.4234, 0.1579, 0.2756],
        [0.2334, 0.8166, 0.8440],
        [0.4211, 0.7514, 0.8699],
        [0.3524, 0.8092, 0.6740],
        [0.4994, 0.1447, 0.2861]])


**Notes:**
- tf_checkpint should include filename like below, rename is necessary or error occurs.
    - :bert_model.ckpt.data-00000-of-00001
    - :bert_model.ckpt.index


In [ ]:
# Run converting.
transformers-cli convert --model_type=bert --tf_checkpoint D:\Xie\Wordpiece_model\checkpoints\convert\bert_model.ckpt --config D:\Xie\Wordpiece_model\checkpoints\convert\config\config.json --pytorch_dump_output D:\Xie\Wordpiece_model\checkpoints\convert\pytorch_model.bin

**Meet errors:**
- Loading TF weight save_counter/.ATTRIBUTES/VARIABLE_VALUE with shape []
Skipping _CHECKPOINTABLE_OBJECT_GRAPH
Traceback (most recent call last):
 -  File "D:\Xie\anaconda3\envs\model_convert\Scripts\transformers-cli-script.py", line 10, in <module>
    sys.exit(main())
 -  File "D:\Xie\anaconda3\envs\model_convert\lib\site-packages\transformers\commands\transformers_cli.py", line 53, in main
    service.run()
 -   File "D:\Xie\anaconda3\envs\model_convert\lib\site-packages\transformers\commands\convert.py", line 104, in run
    convert_tf_checkpoint_to_pytorch(self._tf_checkpoint, self._config, self_pytorch_dump_output)
 -   File "D:\Xie\anaconda3\envs\model_convert\lib\site-packages\transformers\models\bert\convert_bert_original_tf_checkpoint_to_pytorch.py", line 36, in convert_tf_checkpoint_to_pytorch
    load_tf_weights_in_bert(model, config, tf_checkpoint_path)
  File "D:\Xie\anaconda3\envs\model_convert\lib\site-packages\transformers\models\bert\modeling_bert.py", line 156, in load_tf_weights_in_bert
    if pointer.shape != array.shape:
  File "D:\Xie\anaconda3\envs\model_convert\lib\site-packages\torch\nn\modules\module.py", line 594, in __getattr__
    type(self).__name__, name))
AttributeError: 'BertForPreTraining' object has no attribute 'shape'

cd **modeling_bert.py**
- D:\Xie\anaconda3\envs\model_convert\lib\site-packages\transformers\models\bert\modeling_bert.py

Change the lines from 93-174

In [ ]:
# the replace code
def load_tf_weights_in_bert(model, config, tf_checkpoint_path):
    try:
        import re
        import numpy as np
        import tensorflow as tf
    except ImportError:
        logger.error(
            "Loading a TensorFlow model in PyTorch, requires TensorFlow to be installed. Please see "
            "https://www.tensorflow.org/install/ for installation instructions."
        )
        raise
    tf_path = os.path.abspath(tf_checkpoint_path)
    logger.info(f"Converting TensorFlow checkpoint from {tf_path}")
    # Load weights from TF model
    init_vars = tf.train.list_variables(tf_path)
    names = []
    arrays = []
    for name, shape in init_vars:
        logger.info(f"Loading TF weight {name} with shape {shape}")
        array = tf.train.load_variable(tf_path, name)
        names.append(name)
        arrays.append(array)
    for name, array in zip(names, arrays):
        name = name.split("/")
        # adam_v and adam_m are variables used in AdamWeightDecayOptimizer to calculated m and v
        # which are not required for using pretrained model
        if any(
            ["adam_v", "adam_m", "global_step", "bad_steps", "global_step", "good_steps", "loss_scale",
                     "AdamWeightDecayOptimizer", "AdamWeightDecayOptimizer_1", "save_counter", ".OPTIMIZER_SLOT"] for n in name) or \
                name[0] == "optimizer":
        #     n in ["adam_v", "adam_m", "AdamWeightDecayOptimizer", "AdamWeightDecayOptimizer_1", "global_step"]
        #     for n in name
        # ):
            logger.info(f"Skipping {'/'.join(name)}")
            continue
        if ".OPTIMIZER_SLOT" in name:
            idx = name.index(".OPTIMIZER_SLOT")
            name = name[:idx]
        elif ".ATTRIBUTES" in name:
            idx = name.index(".ATTRIBUTES")
            name = name[:idx]
        print(name)
        pointer = model
        for m_name in name:
            if re.fullmatch(r"[A-Za-z]+_\d+", m_name):
                scope_names = re.split(r"_(\d+)", m_name)
            else:
                scope_names = [m_name]
            if scope_names[0] == "kernel" or scope_names[0] == "gamma":
                pointer = getattr(pointer, "weight")
            elif scope_names[0] == "output_bias" or scope_names[0] == "beta":
                pointer = getattr(pointer, "bias")
            elif scope_names[0] == "output_weights":
                pointer = getattr(pointer, "weight")
            elif scope_names[0] == "squad":
                pointer = getattr(pointer, "classifier")
            elif scope_names[0] == "dense_output" or scope_names[0] == "bert_output":
                pointer = getattr(pointer, "output")
            elif scope_names[0] == "self_attention":
                pointer = getattr(pointer, "self")
            else:
                try:
                    pointer = getattr(pointer, scope_names[0])
                except AttributeError:
                    logger.info("Skipping {}".format("/".join(name)))
                    continue
            if len(scope_names) >= 2:
                num = int(scope_names[1])
                pointer = pointer[num]
        if m_name[-11:] == "_embeddings":
            pointer = getattr(pointer, "weight")
        elif m_name == "kernel" or m_name == "gamma" or m_name == "output_weights":
            array = np.transpose(array)
        # try:
        #     if pointer.shape != array.shape:
        #         raise ValueError(f"Pointer shape {pointer.shape} and array shape {array.shape} mismatched")
        # except AssertionError as e:
        #     e.args += (pointer.shape, array.shape)
        #     raise
        logger.info(f"Initialize PyTorch weight {name}")
        pointer.data = torch.from_numpy(array)
    return model

In [ ]:
# run convert again
transformers-cli convert --model_type=bert --tf_checkpoint D:\Xie\Wordpiece_model\checkpoints\convert\bert_model.ckpt --config D:\Xie\Wordpiece_model\checkpoints\convert\config\config.json --pytorch_dump_output D:\Xie\Wordpiece_model\checkpoints\convert\pytorch_model.bin

**Results**
- Skipping model/layer_with_weights-9/_output_layer_norm/beta/.ATTRIBUTES/VARIABLE_VALUE
- Skipping model/layer_with_weights-9/_output_layer_norm/gamma/.ATTRIBUTES/VARIABLE_VALUE
- Skipping save_counter/.ATTRIBUTES/VARIABLE_VALUE
- Save PyTorch model to D:\Xie\Wordpiece_model\checkpoints\convert\pytorch_model.bin

**Referenced:**
- [Huggingface - Converting Tensorflow Checkpoints](https://huggingface.co/docs/transformers/converting_tensorflow_models#t5)
- [Pytorch is not available with Cuda 10.1](https://qiita.com/musicangora/items/efea21b051ecbfd0b500) 
- [BertForPreTraining' object has no attribute 'shape'](https://github.com/huggingface/transformers/issues/393)

